# House Keeping

In [1]:
# Load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, accuracy_score, roc_curve, auc, recall_score, precision_score, f1_score, precision_recall_curve
from sklearn.model_selection import StratifiedKFold,train_test_split,KFold
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import label_binarize
from sklearn import preprocessing

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import Lasso, LinearRegression, Ridge

from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multiclass import OneVsRestClassifier

from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier

from catboost import Pool,CatBoostClassifier
import warnings

import shap
from pingouin import partial_corr

from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.preprocessing import LabelEncoder


In [2]:
# Load the data
df_data = pd.read_csv("C:/Users/Meyke/WiDS/data/train_smogn.csv")
df_test = pd.read_csv("C:/Users/Meyke/WiDS/data/test.csv")

In [3]:
df = pd.concat([df_data, df_test])

In [4]:
# Set energy star rating to np.nan if 1 (since these values seem to be missing)
df['energy_star_rating'] = df.apply(lambda row: np.nan if row.energy_star_rating < 2 else row.energy_star_rating, axis=1)

In [5]:
df.head()

,Unnamed: 0,Year_Factor,State_Factor,building_class,facility_type,floor_area,ELEVATION,january_min_temp,january_avg_temp,january_max_temp,...,days_above_100F,days_above_110F,site_eui,id,year_built,energy_star_rating,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog
0,0.0,4,State_6,Residential,Multifamily_Uncategorized,56543.693554,42.700000,4,28.596774,58,...,0,0,118.453650,41927,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,4,State_6,Residential,Multifamily_Uncategorized,102367.741694,34.755029,4,28.596774,58,...,0,0,119.167517,41949,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,4,State_6,Residential,Multifamily_Uncategorized,159995.763193,12.556233,3,29.466343,62,...,0,0,116.384798,66798,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,4,State_8,Commercial,Public_Assembly_Entertainment_culture,131544.271280,61.744429,3,29.258065,62,...,0,0,119.313113,66819,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,4,State_6,Residential,Multifamily_Uncategorized,407084.764995,31.235663,4,28.596774,58,...,0,0,118.472678,40661,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
original_df_columns = df.columns.to_list()

In [7]:
df_buildings = df[['Year_Factor',
 'State_Factor',
 'building_class',
 'facility_type',
 'floor_area',
 'year_built',
 'energy_star_rating',
 'ELEVATION','id','site_eui']]

In [8]:
df_buildings.head()

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,id,site_eui
0,4,State_6,Residential,Multifamily_Uncategorized,56543.693554,NaN,NaN,42.700000,41927,118.453650
1,4,State_6,Residential,Multifamily_Uncategorized,102367.741694,NaN,NaN,34.755029,41949,119.167517
2,4,State_6,Residential,Multifamily_Uncategorized,159995.763193,NaN,NaN,12.556233,66798,116.384798
3,4,State_8,Commercial,Public_Assembly_Entertainment_culture,131544.271280,NaN,NaN,61.744429,66819,119.313113
4,4,State_6,Residential,Multifamily_Uncategorized,407084.764995,NaN,NaN,31.235663,40661,118.472678


# 1 Quest for Finding Unique Buildings

In [9]:
Find_Unique_building = df.groupby(['State_Factor',
 'building_class',
 'facility_type',
 'floor_area',
 'year_built'])['id'].count().sort_values(ascending=False)

# Create frame
Find_Unique_building = Find_Unique_building.to_frame()
Find_Unique_building.head()

id
State_Factor building_class facility_type             floor_area year_built    
State_4      Residential    Multifamily_Uncategorized 59310.0    1950.0       6
                                                      289267.0   1960.0       5
                                                      107500.0   1952.0       5
                                                      67133.0    1964.0       4
                                                      54096.0    1995.0       4

In [10]:
Find_Unique_building[Find_Unique_building['id']>1]

id
State_Factor building_class facility_type             floor_area year_built    
State_4      Residential    Multifamily_Uncategorized 59310.0    1950.0       6
                                                      289267.0   1960.0       5
                                                      107500.0   1952.0       5
                                                      67133.0    1964.0       4
                                                      54096.0    1995.0       4
                                                      303979.0   1956.0       3
                                                      169000.0   1966.0       2
State_11     Residential    2to4_Unit_Building        35796.0    1968.0       2
State_4      Residential    Multifamily_Uncategorized 64000.0    2007.0       2
                                                      63803.0    2000.0       2
State_11     Residential    2to4_Unit_Building        43228.0    1990.0       2
             Commercial     Office_Uncategorized      25872.0    1969.0       2
State_4      Commercial     Nursing_Home              72844.0    1977.0       2
State_8      Commercial     Retail_Uncategorized      115928.0   2004.0       2
State_2      Residential    Multifamily_Uncategorized 90652.0    1932.0       2
State_11     Residential    2to4_Unit_Building        21200.0    1960.0       2
State_2      Residential    Multifamily_Uncategorized 84000.0    1912.0       2
State_4      Residential    Multifamily_Uncategorized 75600.0    1926.0       2
                                                      63803.0    1999.0       2
             Commercial     Nursing_Home              71586.0    1998.0       2
                            Office_Uncategorized      200150.0   1981.0       2
                            Nursing_Home              60000.0    1996.0       2
             Residential    Multifamily_Uncategorized 55000.0    1976.0       2
State_11     Residential    5plus_Unit_Building       21294.0    1991.0       2
State_8      Commercial     Office_Uncategorized      248000.0   1983.0       2
State_11     Residential    2to4_Unit_Building        22625.0    2004.0       2

In [11]:
Find_Unique_building = Find_Unique_building.reset_index()

In [12]:
Find_Unique_building = Find_Unique_building[Find_Unique_building['id']>1]

In [13]:
Find_Unique_building.head()

,State_Factor,building_class,facility_type,floor_area,year_built,id
0,State_4,Residential,Multifamily_Uncategorized,59310.0,1950.0,6
1,State_4,Residential,Multifamily_Uncategorized,289267.0,1960.0,5
2,State_4,Residential,Multifamily_Uncategorized,107500.0,1952.0,5
3,State_4,Residential,Multifamily_Uncategorized,67133.0,1964.0,4
4,State_4,Residential,Multifamily_Uncategorized,54096.0,1995.0,4


In [14]:
# Lets use the new index to Find_Unique_building to assign to rows that seem to apply to the same building, in a new column named "building_id"
df_buildings['building_id'] = df_buildings.apply(lambda row: np.nan if Find_Unique_building[(Find_Unique_building['State_Factor'] == row.State_Factor) & 
                     (Find_Unique_building['building_class'] == row.building_class) & 
                     (Find_Unique_building['facility_type'] == row.facility_type) & 
                     (Find_Unique_building['floor_area'] == row.floor_area) & 
                     (Find_Unique_building['year_built'] == row.year_built)].empty else Find_Unique_building[(Find_Unique_building['State_Factor'] == row.State_Factor) & 
                     (Find_Unique_building['building_class'] == row.building_class) & 
                     (Find_Unique_building['facility_type'] == row.facility_type) & 
                     (Find_Unique_building['floor_area'] == row.floor_area) & 
                     (Find_Unique_building['year_built'] == row.year_built)].index.values[0], axis=1)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [15]:
# Check correct execution
df_buildings[df_buildings['building_id'].notna()]

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,id,site_eui,building_id
1593,7,State_4,Commercial,Office_Uncategorized,200150.0,1981.0,70.0,260.0,77350,NaN,20.0
1748,7,State_8,Commercial,Office_Uncategorized,248000.0,1983.0,87.0,130.8,77505,NaN,24.0
1763,7,State_8,Commercial,Office_Uncategorized,248000.0,1983.0,89.0,130.8,77520,NaN,24.0
2627,7,State_4,Commercial,Nursing_Home,60000.0,1996.0,42.0,241.4,78384,NaN,21.0
2628,7,State_4,Commercial,Nursing_Home,60000.0,1996.0,38.0,229.8,78385,NaN,21.0
...,...,...,...,...,...,...,...,...,...,...,...
8932,7,State_11,Residential,2to4_Unit_Building,43228.0,1990.0,100.0,57.3,84689,NaN,10.0
9272,7,State_11,Residential,2to4_Unit_Building,43228.0,1990.0,100.0,57.3,85029,NaN,10.0
9281,7,State_11,Residential,2to4_Unit_Building,21200.0,1960.0,82.0,33.5,85038,NaN,15.0
9492,7,State_11,Residential,5plus_Unit_Building,21294.0,1991.0,89.0,33.5,85249,NaN,23.0


In [16]:
df_buildings.head()

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,id,site_eui,building_id
0,4,State_6,Residential,Multifamily_Uncategorized,56543.693554,NaN,NaN,42.700000,41927,118.453650,NaN
1,4,State_6,Residential,Multifamily_Uncategorized,102367.741694,NaN,NaN,34.755029,41949,119.167517,NaN
2,4,State_6,Residential,Multifamily_Uncategorized,159995.763193,NaN,NaN,12.556233,66798,116.384798,NaN
3,4,State_8,Commercial,Public_Assembly_Entertainment_culture,131544.271280,NaN,NaN,61.744429,66819,119.313113,NaN
4,4,State_6,Residential,Multifamily_Uncategorized,407084.764995,NaN,NaN,31.235663,40661,118.472678,NaN


In [17]:
df['building_id'] = df_buildings['building_id'].to_list()

In [18]:
df.head()

,Unnamed: 0,Year_Factor,State_Factor,building_class,facility_type,floor_area,ELEVATION,january_min_temp,january_avg_temp,january_max_temp,...,days_above_110F,site_eui,id,year_built,energy_star_rating,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,building_id
0,0.0,4,State_6,Residential,Multifamily_Uncategorized,56543.693554,42.700000,4,28.596774,58,...,0,118.453650,41927,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,4,State_6,Residential,Multifamily_Uncategorized,102367.741694,34.755029,4,28.596774,58,...,0,119.167517,41949,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,4,State_6,Residential,Multifamily_Uncategorized,159995.763193,12.556233,3,29.466343,62,...,0,116.384798,66798,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,4,State_8,Commercial,Public_Assembly_Entertainment_culture,131544.271280,61.744429,3,29.258065,62,...,0,119.313113,66819,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,4,State_6,Residential,Multifamily_Uncategorized,407084.764995,31.235663,4,28.596774,58,...,0,118.472678,40661,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df[['building_id']].isna().sum()

building_id    75620
dtype: int64

In [20]:
df['building_id'] = df.apply(lambda row:  row.building_id if row.building_id == row.building_id else
                             df['building_id'].max() + row.id, axis=1)

In [21]:
le = LabelEncoder()
df['building_id'] = le.fit_transform(df['building_id'])

In [22]:
df[df['building_id']==0]

,Unnamed: 0,Year_Factor,State_Factor,building_class,facility_type,floor_area,ELEVATION,january_min_temp,january_avg_temp,january_max_temp,...,days_above_110F,site_eui,id,year_built,energy_star_rating,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,building_id
3112,NaN,7,State_4,Residential,Multifamily_Uncategorized,59310.0,186.5,-1,27.548387,59,...,0,NaN,78869,1950.0,59.0,340.0,320.0,19.2,NaN,0
3221,NaN,7,State_4,Residential,Multifamily_Uncategorized,59310.0,186.5,-1,27.548387,59,...,0,NaN,78978,1950.0,4.0,340.0,320.0,19.2,NaN,0
3240,NaN,7,State_4,Residential,Multifamily_Uncategorized,59310.0,186.5,-1,27.548387,59,...,0,NaN,78997,1950.0,18.0,340.0,320.0,19.2,NaN,0
8017,NaN,7,State_4,Residential,Multifamily_Uncategorized,59310.0,186.5,-1,27.548387,59,...,0,NaN,83774,1950.0,26.0,340.0,320.0,19.2,NaN,0
8294,NaN,7,State_4,Residential,Multifamily_Uncategorized,59310.0,186.5,-1,27.548387,59,...,0,NaN,84051,1950.0,31.0,340.0,320.0,19.2,NaN,0
8440,NaN,7,State_4,Residential,Multifamily_Uncategorized,59310.0,186.5,-1,27.548387,59,...,0,NaN,84197,1950.0,58.0,340.0,320.0,19.2,NaN,0


In [23]:
df.head()

,Unnamed: 0,Year_Factor,State_Factor,building_class,facility_type,floor_area,ELEVATION,january_min_temp,january_avg_temp,january_max_temp,...,days_above_110F,site_eui,id,year_built,energy_star_rating,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,building_id
0,0.0,4,State_6,Residential,Multifamily_Uncategorized,56543.693554,42.700000,4,28.596774,58,...,0,118.453650,41927,NaN,NaN,NaN,NaN,NaN,NaN,27109
1,1.0,4,State_6,Residential,Multifamily_Uncategorized,102367.741694,34.755029,4,28.596774,58,...,0,119.167517,41949,NaN,NaN,NaN,NaN,NaN,NaN,27125
2,2.0,4,State_6,Residential,Multifamily_Uncategorized,159995.763193,12.556233,3,29.466343,62,...,0,116.384798,66798,NaN,NaN,NaN,NaN,NaN,NaN,43278
3,3.0,4,State_8,Commercial,Public_Assembly_Entertainment_culture,131544.271280,61.744429,3,29.258065,62,...,0,119.313113,66819,NaN,NaN,NaN,NaN,NaN,NaN,43293
4,4.0,4,State_6,Residential,Multifamily_Uncategorized,407084.764995,31.235663,4,28.596774,58,...,0,118.472678,40661,NaN,NaN,NaN,NaN,NaN,NaN,26176


In [24]:
df_B_id_count = df.groupby(['building_id'])['building_id'].count().sort_values(ascending=False).to_frame()

In [25]:
df_B_id_count = df_B_id_count.rename(columns={"building_id": "count"})

In [26]:
df_B_id_count.groupby(['count'])['count'].count()

count
1    44497
2    10932
3     2275
4      459
5      101
6       22
7        4
Name: count, dtype: int64

# 3 impute energy_star_rating

In [27]:
building_id_list = df['building_id'].unique()

In [28]:
import math

def impute_star(row):
    if row.energy_star_rating == row.energy_star_rating:
        return row.energy_star_rating
    else:
        B_id = row.building_id
        B_year = row.Year_Factor
        
        temp_df = df[df['building_id'] == B_id] 
        
        # list_star is list of energy_star_ratings
        list_star = temp_df['energy_star_rating'].to_list()
        
        # list_year is list of year_factor
        list_year = temp_df['Year_Factor'].to_list()
        
        # list of the indexes of the nan values in list_star
        nan_list = [list_star.index(x) for x in list_star if math.isnan(x) == True]
        len_nan_list = len(nan_list)
        #print("nan_list", nan_list)
        
        if len_nan_list > 0:
            # neem index 0 van len_nan_list
            smallest_nan_value = nan_list[0]
            #print("smallest:", smallest_nan_value)
            # neem laatste index van len_nan_list
            largest_nan_value = nan_list[-1]
            #print("largest:", largest_nan_value)
        
            # check of er kleinere waarde in de list_star zitten, neem die value, het gemiddelde van die values of np.nan als er geen zijn
            if smallest_nan_value > 0:
                #print("smaller than 0")
                get_year = list_year[smallest_nan_value - 1]
                # check if the previous year is a duplicate year
                duplicate_year = list_year.count(get_year) > 1
                if duplicate_year:
                    # get indexes of the duplicate year
                    get_indices_years = [i for i,val in enumerate(list_year) if val==get_year]
                    # calc the average of the star ratings
                    temp_previous_year = 0
                    for z in get_indices_years:
                        temp_previous_year += list_star[z]
                    temp_previous_year = temp_previous_year/z
                else:
                    temp_previous_year = list_star[list_year.index(get_year)]
            else:
                #print("larger than 0")
                temp_previous_year = np.nan

            # check of er grotere waarde in de list_star zitten, neem die value, het gemiddelde van die values of np.nan als er geen zijn
            if largest_nan_value+1 < len(list_star):
                #print("smaller than length of the list", largest_nan_value)
                get_year = list_year[largest_nan_value + 1]
                #print("list_year", list_year)
                #print("get_year", get_year)
                #print("star_list", list_star)
                # check if the next year is a duplicate year
                duplicate_year = list_year.count(get_year) > 1
                if duplicate_year:
                    # get indexes of the duplicate year
                    get_indices_years = [i for i,val in enumerate(list_year) if val==get_year]
                    # calc the average of the star ratings
                    temp_next_year = 0
                    for z in get_indices_years:
                        temp_next_year += list_star[z]
                    temp_next_year = temp_next_year/z
                else:
                    temp_next_year = list_star[list_year.index(get_year)]

            else:
                temp_next_year = np.nan


            if (temp_next_year == temp_next_year) & (temp_previous_year ==  temp_previous_year):
                return (temp_next_year+temp_previous_year)/2
            elif (temp_next_year == temp_next_year) & (temp_previous_year != temp_previous_year):
                return temp_next_year
            elif (temp_next_year != temp_next_year) & (temp_previous_year == temp_previous_year):
                return temp_previous_year
            else:
                return np.nan

In [29]:
df['star'] = df.apply(impute_star, axis=1)

In [30]:
# almost 30061K rows imputed
df[df['star'].notna()]

,Unnamed: 0,Year_Factor,State_Factor,building_class,facility_type,floor_area,ELEVATION,january_min_temp,january_avg_temp,january_max_temp,...,site_eui,id,year_built,energy_star_rating,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,building_id,star
28103,0.0,1,State_1,Commercial,Grocery_store_or_food_market,61242.0,2.4,36,50.500000,68,...,248.682615,0,NaN,NaN,NaN,NaN,NaN,NaN,25,90.5
0,NaN,7,State_1,Commercial,Grocery_store_or_food_market,28484.0,2.4,38,50.596774,64,...,NaN,75757,1994.0,37.0,NaN,NaN,NaN,NaN,48652,37.0
1,NaN,7,State_1,Commercial,Grocery_store_or_food_market,21906.0,45.7,38,50.596774,64,...,NaN,75758,1961.0,55.0,NaN,NaN,NaN,NaN,48653,55.0
3,NaN,7,State_1,Commercial,Grocery_store_or_food_market,97422.0,35.4,38,50.596774,64,...,NaN,75760,1971.0,34.0,NaN,NaN,NaN,NaN,48655,34.0
4,NaN,7,State_1,Commercial,Grocery_store_or_food_market,61242.0,1.8,38,50.596774,64,...,NaN,75761,1942.0,35.0,340.0,330.0,22.8,126.0,48656,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9700,NaN,7,State_11,Residential,5plus_Unit_Building,305925.0,812.0,21,38.661290,54,...,NaN,85457,2013.0,99.0,NaN,NaN,NaN,NaN,58285,99.0
9701,NaN,7,State_11,Residential,5plus_Unit_Building,34909.0,26.5,21,38.661290,54,...,NaN,85458,1910.0,94.0,NaN,NaN,NaN,NaN,58286,94.0
9702,NaN,7,State_11,Residential,5plus_Unit_Building,42393.0,812.0,21,38.661290,54,...,NaN,85459,2014.0,94.0,NaN,NaN,NaN,NaN,58287,94.0
9703,NaN,7,State_11,Residential,5plus_Unit_Building,25368.0,812.0,21,38.661290,54,...,NaN,85460,1992.0,79.0,NaN,NaN,NaN,NaN,58288,79.0


In [31]:
missing_site_and_star = df[(df['Year_Factor']==7)&(df['star'].isna())].groupby(['building_class','facility_type'])['facility_type'].count()
missing_site_and_star

building_class  facility_type                            
Commercial      Commercial_Other                             316
                Commercial_Unknown                            21
                Data_Center                                    3
                Education_College_or_university              201
                Education_Other_classroom                     31
                Education_Preschool_or_daycare                 3
                Education_Uncategorized                       11
                Food_Sales                                     2
                Food_Service_Other                             3
                Food_Service_Restaurant_or_cafeteria          17
                Food_Service_Uncategorized                     4
                Grocery_store_or_food_market                  10
                Health_Care_Inpatient                         15
                Health_Care_Outpatient_Clinic                  5
                Health_Care_Outp

In [32]:
def impute_missing_star(row):
    if row.star == row.star:
        return row.star
    else:
        return df[df['facility_type']== row.facility_type]['energy_star_rating'].median()

In [33]:
df['star'] = df.apply(impute_missing_star, axis=1)

In [34]:
del df['energy_star_rating']

In [35]:
df = df.rename(columns={"star": "energy_star_rating"})

# 4 Create other wind columns

In [36]:
df['peak_NE'] = df.apply(lambda row: 1 if (row.direction_peak_wind_speed < 91) & (row.direction_peak_wind_speed > 0) else 0, axis=1)
df['peak_SE'] = df.apply(lambda row: 1 if (row.direction_peak_wind_speed < 181) & (row.direction_peak_wind_speed > 90) else 0, axis=1)
df['peak_SW'] = df.apply(lambda row: 1 if (row.direction_peak_wind_speed < 271) & (row.direction_peak_wind_speed > 180) else 0, axis=1)
df['peak_NW'] = df.apply(lambda row: 1 if (row.direction_peak_wind_speed < 361) & (row.direction_peak_wind_speed > 270) else 0, axis=1)

df['max_NE'] = df.apply(lambda row: 1 if (row.direction_max_wind_speed < 91) & (row.direction_max_wind_speed > 0) else 0, axis=1)
df['max_SE'] = df.apply(lambda row: 1 if (row.direction_max_wind_speed < 181) & (row.direction_max_wind_speed > 90) else 0, axis=1)
df['max_SW'] = df.apply(lambda row: 1 if (row.direction_max_wind_speed < 271) & (row.direction_max_wind_speed > 180) else 0, axis=1)
df['max_NW'] = df.apply(lambda row: 1 if (row.direction_max_wind_speed < 361) & (row.direction_max_wind_speed > 270) else 0, axis=1)

In [37]:
del df['direction_max_wind_speed']
del df['direction_peak_wind_speed']

# 5 fill na for days with fog and max wind speed

In [38]:
df['days_with_fog'] = df['days_with_fog'].fillna(0)

In [39]:
df['max_wind_speed'] = df['max_wind_speed'].fillna(-1)

In [40]:
df[df['energy_star_rating'].isna()][['State_Factor','Year_Factor','floor_area','facility_type','building_id','site_eui']]

,State_Factor,Year_Factor,floor_area,facility_type,building_id,site_eui
3,State_8,4,1.315443e+05,Public_Assembly_Entertainment_culture,43293,119.313113
26,State_6,6,4.741135e+04,Public_Safety_Fire_or_police_station,41299,128.122115
27,State_6,6,2.228793e+04,Public_Safety_Fire_or_police_station,41290,118.971034
96,State_4,6,1.468219e+05,Retail_Enclosed_mall,8588,123.175888
97,State_4,6,1.857208e+05,Retail_Enclosed_mall,8787,158.335108
...,...,...,...,...,...,...
8792,State_2,7,6.811600e+04,Lodging_Other,57384,NaN
8796,State_4,7,4.570176e+06,Public_Safety_Penitentiary,57388,NaN
8797,State_8,7,1.600000e+05,Public_Safety_Penitentiary,57389,NaN
8798,State_8,7,4.362500e+05,Public_Safety_Penitentiary,57390,NaN


# 6 Create facility_group column

In [41]:
#create facility groups
food = ['Grocery_store_or_food_market', 'Food_Service_Uncategorized', 'Food_Service_Other', 'Food_Service_Restaurant_or_cafeteria',
        'Food_Sales']

distribution_shipping = ['Warehouse_Distribution_or_Shipping_center', 'Warehouse_Nonrefrigerated',  'Warehouse_Selfstorage',
                         'Warehouse_Refrigerated',  'Warehouse_Uncategorized']

retail = ['Retail_Enclosed_mall', 'Retail_Vehicle_dealership_showroom' , 'Retail_Uncategorized',  'Retail_Strip_shopping_mall']

eduation = ['Education_Other_classroom', 'Education_College_or_university', 'Education_Uncategorized', 'Education_Preschool_or_daycare']

office = ['Office_Uncategorized', 'Office_Mixed_use']

technology = ['Data_Center', 'Laboratory']

other = ['Commercial_Other', 'Mixed_Use_Predominantly_Commercial', 'Mixed_Use_Commercial_and_Residential', 'Commercial_Unknown']

other_res = ['Mixed_Use_Predominantly_Residential',  '2to4_Unit_Building', '5plus_Unit_Building']

health_care = ['Office_Medical_non_diagnostic', 'Health_Care_Inpatient', 'Health_Care_Uncategorized', 'Health_Care_Outpatient_Clinic',
               'Nursing_Home', 'Health_Care_Outpatient_Uncategorized']

industrial = ['Industrial']

public_assembly = ['Public_Assembly_Entertainment_culture', 'Public_Assembly_Drama_theater', 'Public_Assembly_Social_meeting',
                   'Public_Assembly_Recreation', 'Public_Assembly_Movie_Theater', 'Public_Assembly_Uncategorized',
                   'Public_Assembly_Other', 'Public_Assembly_Stadium']

public_service = ['Public_Assembly_Library', 'Public_Safety_Uncategorized', 'Public_Safety_Fire_or_police_station',
                  'Public_Safety_Penitentiary', 'Public_Safety_Courthouse']

lodging_residential = ['Lodging_Hotel',  'Multifamily_Uncategorized', 'Lodging_Dormitory_or_fraternity_sorority', 'Lodging_Other', 'Lodging_Uncategorized']

worship = ['Religious_worship']

banking = ['Office_Bank_or_other_financial']

parking = ['Parking_Garage']

services = [ 'Service_Vehicle_service_repair_shop', 'Service_Drycleaning_or_Laundry', 'Service_Uncategorized']

In [42]:
#apply grouping
df['facility_group'] = 1

df.loc[df['facility_type'].isin(food), "facility_group"] = 'food_sales_service'  
df.loc[df['facility_type'].isin(distribution_shipping), "facility_group"] = 'distribution_shipping'  
df.loc[df['facility_type'].isin(retail), "facility_group"] = 'retail'  
df.loc[df['facility_type'].isin(eduation), "facility_group"] = 'eduation'  
df.loc[df['facility_type'].isin(office), "facility_group"] = 'office'  
df.loc[df['facility_type'].isin(technology), "facility_group"] = 'technology'  
df.loc[df['facility_type'].isin(other), "facility_group"] = 'other' 
df.loc[df['facility_type'].isin(health_care), "facility_group"] = 'health_care'
df.loc[df['facility_type'].isin(industrial), "facility_group"] = 'industrial'    
df.loc[df['facility_type'].isin(public_assembly ), "facility_group"] = 'public_assembly'  
df.loc[df['facility_type'].isin(public_service ), "facility_group"] = 'public_service'  
df.loc[df['facility_type'].isin(lodging_residential), "facility_group"] = 'lodging_residential'  
df.loc[df['facility_type'].isin(worship), "facility_group"] = 'worship'  
df.loc[df['facility_type'].isin(banking), "facility_group"] = 'banking'  
df.loc[df['facility_type'].isin(parking), "facility_group"] = 'parking'  
df.loc[df['facility_type'].isin(services), "facility_group"] = 'services'  
df.loc[df['facility_type'].isin(other_res), "facility_group"] = 'other_res' 

df.head()

,Unnamed: 0,Year_Factor,State_Factor,building_class,facility_type,floor_area,ELEVATION,january_min_temp,january_avg_temp,january_max_temp,...,energy_star_rating,peak_NE,peak_SE,peak_SW,peak_NW,max_NE,max_SE,max_SW,max_NW,facility_group
0,0.0,4,State_6,Residential,Multifamily_Uncategorized,56543.693554,42.700000,4,28.596774,58,...,63.0,0,0,0,0,0,0,0,0,lodging_residential
1,1.0,4,State_6,Residential,Multifamily_Uncategorized,102367.741694,34.755029,4,28.596774,58,...,63.0,0,0,0,0,0,0,0,0,lodging_residential
2,2.0,4,State_6,Residential,Multifamily_Uncategorized,159995.763193,12.556233,3,29.466343,62,...,63.0,0,0,0,0,0,0,0,0,lodging_residential
3,3.0,4,State_8,Commercial,Public_Assembly_Entertainment_culture,131544.271280,61.744429,3,29.258065,62,...,NaN,0,0,0,0,0,0,0,0,public_assembly
4,4.0,4,State_6,Residential,Multifamily_Uncategorized,407084.764995,31.235663,4,28.596774,58,...,63.0,0,0,0,0,0,0,0,0,lodging_residential


# 7 Impute last missing Energy star ratings based on facility group column

In [43]:
def impute_missing_star_2(row):
    if row.energy_star_rating == row.energy_star_rating:
        return row.energy_star_rating
    else:
        return df[df['facility_group']== row.facility_group]['energy_star_rating'].median()

In [55]:
df['energy_star_rating'] = df.apply(impute_missing_star_2, axis=1)

In [56]:
df['energy_star_rating'].isna().sum()

1573

In [54]:
df[df['energy_star_rating'].isna()]

,Unnamed: 0,Year_Factor,State_Factor,building_class,facility_type,floor_area,ELEVATION,january_min_temp,january_avg_temp,january_max_temp,...,energy_star_rating,peak_NE,peak_SE,peak_SW,peak_NW,max_NE,max_SE,max_SW,max_NW,facility_group
3,3.0,4,State_8,Commercial,Public_Assembly_Entertainment_culture,1.315443e+05,61.744429,3,29.258065,62,...,NaN,0,0,0,0,0,0,0,0,public_assembly
228,228.0,1,State_6,Commercial,Public_Assembly_Other,6.414919e+04,20.115558,6,29.677419,53,...,NaN,0,0,0,0,0,0,0,0,public_assembly
229,229.0,1,State_6,Commercial,Public_Assembly_Other,6.483766e+04,17.803195,6,29.677419,53,...,NaN,0,0,0,0,0,0,0,0,public_assembly
361,361.0,5,State_4,Commercial,Public_Assembly_Recreation,6.800931e+04,214.264754,0,21.935484,46,...,NaN,0,0,0,0,0,0,0,0,public_assembly
408,408.0,4,State_1,Commercial,Public_Assembly_Entertainment_culture,1.213598e+05,9.100000,41,55.096774,73,...,NaN,0,0,0,0,0,0,0,0,public_assembly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7520,NaN,7,State_4,Commercial,Public_Assembly_Recreation,5.127300e+04,214.000000,-1,27.548387,59,...,NaN,0,0,0,0,0,0,0,0,public_assembly
7617,NaN,7,State_1,Commercial,Service_Drycleaning_or_Laundry,1.360800e+04,43.900000,38,50.596774,64,...,NaN,0,0,0,0,0,0,0,0,services
8738,NaN,7,State_2,Commercial,Public_Assembly_Other,2.000000e+05,56.100000,15,41.177419,71,...,NaN,0,0,0,0,0,0,0,0,public_assembly
8739,NaN,7,State_2,Commercial,Public_Assembly_Other,2.300000e+06,15.200000,15,41.177419,71,...,NaN,0,0,0,0,0,0,0,0,public_assembly


In [59]:
df['energy_star_rating'] = df['energy_star_rating'].fillna(-1)

# 8 Impute Year Built

In [60]:
df[df['year_built']==0]

,Unnamed: 0,Year_Factor,State_Factor,building_class,facility_type,floor_area,ELEVATION,january_min_temp,january_avg_temp,january_max_temp,...,energy_star_rating,peak_NE,peak_SE,peak_SW,peak_NW,max_NE,max_SE,max_SW,max_NW,facility_group


In [61]:
#set year 0 to np.nan
df['year_built'] = np.where(df['year_built']==0, np.nan, df['year_built'])

In [62]:
#create train and test
df_train= df[df["id"].isin(df_data["id"])]
df_test = df[df["id"].isin(df_test["id"])]

In [64]:
del df_train['Unnamed: 0']

In [66]:
del df_test['Unnamed: 0']

In [65]:
df_test.isna().sum().sort_values(ascending=False)

Unnamed: 0          9705
site_eui            9705
year_built            93
snowdepth_inches       0
days_below_0F          0
                    ... 
june_max_temp          0
june_avg_temp          0
june_min_temp          0
may_max_temp           0
facility_group         0
Length: 73, dtype: int64

In [67]:
#drop columns that we won't include in mice imputing
mice_df_train = df_train .drop(['site_eui', 'id','peak_NE', 'peak_SE', 'peak_SW', 'peak_NW',
       'max_NE', 'max_SE', 'max_SW', 'max_NW' ],axis=1) 
mice_df_test = df_test.drop(['site_eui' , 'id','peak_NE', 'peak_SE', 'peak_SW', 'peak_NW',
       'max_NE', 'max_SE', 'max_SW', 'max_NW' ],axis=1) 

In [68]:
#create a list of all categorical features
categorical_cols = [cname for cname in mice_df_train.columns if
                    mice_df_train [cname].dtype == "object"]

# Encode Categorical Columns 
for col in categorical_cols:
    le = LabelEncoder()
    mice_df_train [col] = le.fit_transform(mice_df_train[col])
    mice_df_test[col] = le.transform(mice_df_test[col])

In [69]:
#create a list of the maximum values of each column
max_list = []
for col in mice_df_train.columns:
    max_value = mice_df_train[col].max()
    max_list.append(max_value)

#create a list of the minimum values of each column
min_list = []
for col in mice_df_train.columns:
    min_value = mice_df_train[col].min()
    min_list.append(min_value)

In [ ]:
mice_df_train

In [ ]:
mice_df_test

In [ ]:
mice = mice_df_train.append(mice_df_test)
mice['id'] = df['id']
mice

In [ ]:
mice= mice[['id', 'year_built']]
mice = mice.sort_values('id')
mice

In [ ]:
df_imputed = df.merge(mice[['id','year_built']], on = 'id', how = 'left' )

In [ ]:
df_imputed

In [ ]:
df_imputed.isna().sum().sort_values(ascending=False)

In [ ]:
df_imputed[df_imputed['year_built_x'] != df_imputed['year_built_y']]

In [ ]:
del df_imputed['year_built_x']

In [ ]:
#del df_imputed['energy_star_rating_y']

In [ ]:
df = df_imputed.rename(columns={"year_built_y": "year_built", "Year_Factor": "year_factor", "State_Factor": "state_factor", "ELEVATION": "elevation"})

In [ ]:
df.head()

In [ ]:
df.isna().sum().sort_values(ascending=False)

In [ ]:
df.to_csv("C:/Users/Meyke/WiDS/data/clean_data_3001_imputed_star_rating_and_generous_building_id.csv")